In [14]:
import pandas as pd 
import geopandas as gpd
pd.options.mode.chained_assignment = None  # default='warn'

On fait tourner ça et ça se met à jour tous les jours 

In [15]:
import urllib
urllib.request.urlretrieve("https://data.economie.gouv.fr/explore/dataset/prix-carburants-fichier-quotidien-test-ods/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B", "data/fichier.csv")

('data/fichier.csv', <http.client.HTTPMessage at 0x1a4368fc7f0>)

In [16]:
urllib.request.urlretrieve("https://data.economie.gouv.fr/explore/dataset/prix-carburants-fichier-instantane-test-ods-copie/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B", "data/fichier_inst.csv")

('data/fichier_inst.csv', <http.client.HTTPMessage at 0x1a4368f7ca0>)

In [17]:
dat =pd.read_csv('data/fichier.csv', sep= ';', on_bad_lines= 'skip')

In [18]:
dat_inst = pd.read_csv('data/fichier_inst.csv', sep= ';', on_bad_lines= 'skip')

In [19]:
data = pd.read_csv('data/prix-carburants-fichier-quotidien-test-ods.csv', sep= ';', on_bad_lines= 'skip')

In [20]:
data.columns

Index(['id', 'cp', 'pop', 'adresse', 'ville', 'horaires', 'rupture',
       'fermeture', 'geom', 'prix_maj', 'prix_id', 'prix_valeur', 'prix_nom',
       'com_arm_code', 'Nom Officiel Commune / Arrondissement Municipal',
       'Code Officiel EPCI', 'Nom Officiel EPCI', 'Code Officiel Département',
       'Nom Officiel Département', 'Code Officiel Région',
       'Nom Officiel Région', 'services_service', 'Nom rupture',
       'Début rupture', 'Fin rupture', 'Automate 24-24 (oui/non)'],
      dtype='object')

Fonction pour définir la "bonne" base de données avec les données qui nous intéressent  
Il faut l'améliorer pour qu'elle élimine les doublons

In [23]:
def data_carb(df, carb):
    data_1 = df[['adresse', 'ville', 'geom', 'prix_valeur', 'prix_nom']]
    d = data_1[data_1.prix_nom == carb]
    d.drop_duplicates(keep = 'first', inplace=True) 
    #df.drop_duplicates(subset ="'column'", keep = 'first', inplace=True) si on veut supprimer selon une colonne
    return d




In [24]:
data_2 = data_carb(data, 'SP95').sort_values('prix_valeur')

Fonction pour calculer la distance à partir de données de longitude et de latitude

In [25]:
from math import sin, cos, acos, pi
 
def dms2dd(d, m, s):
    """Convertit un angle "degrés minutes secondes" en "degrés décimaux"
    """
    return d + m/60 + s/3600
 
def dd2dms(dd):
    """Convertit un angle "degrés décimaux" en "degrés minutes secondes"
    """
    d = int(dd)
    x = (dd-d)*60
    m = int(x)
    s = (x-m)*60
    return d, m, s
 
def deg2rad(dd):
    """Convertit un angle "degrés décimaux" en "radians"
    """
    return dd/180*pi
 

def rad2deg(rd):
    """Convertit un angle "radians" en "degrés décimaux"
    """
    return rd/pi*180
 
#############################################################################
def distanceGPS(p1, p2):
    """Retourne la distance en mètres entre les 2 points A et B connus grâce à
       leurs coordonnées GPS (en radians).
    """
    latA, longA = p1
    latB, longB = p2
    # Rayon de la terre en mètres (sphère IAG-GRS80)
    RT = 6378137
    # angle en radians entre les 2 points
    S = acos(sin(latA)*sin(latB) + cos(latA)*cos(latB)*cos(abs(longB-longA)))
    # distance entre les 2 points, comptée sur un arc de grand cercle
    return S*RT




In [26]:
from geopy.geocoders import Nominatim


def Adresse_to_gps(Adresse) : 

    geolocator = Nominatim(user_agent="Projet Python")
    location = geolocator.geocode(Adresse)
    return str(str(location.latitude) + ','+ str(location.longitude))


Adresse_to_gps("1 rue d'Arsonval, Orsay, France")

'48.7087952,2.1742033'

Fonction pour renvoyer un data frame avec les 10 plus proches stations essences qui propose le type de carburant recherché   
Majeur problème : les doublons 

In [27]:
def dix_plus_proche_card(df, posi, carb):
    data_1 = data_carb(df, carb).sort_values('prix_valeur')
    pos = list(map(float,Adresse_to_gps(posi).split(',')))
    def distancetoposi(p2):
        pos_carb = list(map(float, p2.split(',')))
        return distanceGPS(pos, pos_carb)
    new_df = data_1['geom'].apply(distancetoposi).sort_values().head(10)
    return data_1.loc[new_df.index.to_list()]



    


In [28]:
dix_plus_proche_card(dat_inst, "1 rue d'Arsonval, Orsay, France" , 'SP95')

,adresse,ville,geom,prix_valeur,prix_nom
1138,69 Avenue Charles de Gaulle,MORANGIS,"48.706,2.343",1.739,SP95
3873,Rue de la Ballastière,Ligny-en-Barrois,"48.677,5.326",1.631,SP95
28782,180 ROUTE NATIONALE 7,ATHIS-MONS,"48.712,2.371",1.619,SP95
19739,KERGUENNEC,ROSCOFF,"48.71344,-3.98569",1.750,SP95
3542,lieudit le village,Les Essarts-le-Roi,"48.71419,1.92645",1.689,SP95
17396,RUE DU GENERAL DE GAULLE,AVRANCHES,"48.68647,-1.36356",1.769,SP95
9728,PLACE DE BEAUPLAN,SAINT-RéMY-LèS-CHEVREUSE,"48.716,2.085",1.737,SP95
30751,72 Avenue Jean Jaurès,Montgeron,"48.716,2.446",1.740,SP95
26596,118 Rue de la Passerelle,CHILLY-MAZARIN,"48.69943,2.32057",1.799,SP95
4112,120A ROUTE NATIONALE,GAMBSHEIM,"48.699248714218,7.889266294415299",1.559,SP95
